In [1]:
import json
import numpy as np
import pandas as pd

# Hospital Shift Manager
Building the backend of an hospital shift manager application.

This assignment is taken from the DSGT Bootcamp assignment 6 in section 4.

"Hi. I hope you are doing well. My team would like to develop a hospital shift manager application. We have some frontend designers (ie: UI/UX people), but we need your help as a backend developer. To make our application work, we need you to help us work with our JSON data on the shifts. NOTE that data.json and example output.json files are just examples for sake of formatting. Don't rely on these to "evaluate the correctness of your solution". It is your responsibility to test your implementation on some test cases. Your submission should clearly identify your code specific to Task 1, Task 2, Task 3, Task 4"

## Task 1
A worker is either a medic or intern. Per shift, a medic is paid USD305 while an intern is paid USD142. Build an endpoint that takes in a JSON file called data.json and produces a file called output.json that is formatted as the following.

In [2]:
shift_rates = {"medic": 305, "intern": 142}

def retrieve_personel_id_pay(json_file):
    # Get the data in input.json (as a dictionary)
    f = open(json_file)
    data = json.load(f)
    f.close()

    # Convert data["workers"] into a dataframe
    df = pd.DataFrame(data["workers"])
    shifts = pd.DataFrame(data["shifts"])
    
    # Add new "Work dates" and "pay" columns
    df["Work dates"] = np.nan
    df["pay"] = 0
    
    for index1, row1 in df.iterrows():
        staff_id = row1["id"]
        for index2, row2 in shifts.iterrows():
            if row2["user_id"] == staff_id:
                if df.isna().loc[index1, "Work dates"]:
                    df.loc[index1, "Work dates"] = [row2["start_date"]]
                else:
                    df.loc[index1, "Work dates"].append(row2["start_date"])
        df.loc[index1,"pay"] = shift_rates[row1["status"]] * len(df.loc[index1, "Work dates"])
        
    # Filter the columns of interest
    df = df[["id", "pay"]]
    
    # Convert data back into a dictionary
    output_dict = {}
    output_dict["workers"] = df.to_dict("records")
    
    # Convert data back into a json file
    with open("output.json", "w") as f:
        json.dump(output_dict, f)
    
    return json.dumps(output_dict)

retrieve_personel_id_pay("input.json")

'{"workers": [{"id": 1, "pay": 915}, {"id": 2, "pay": 915}, {"id": 3, "pay": 284}, {"id": 4, "pay": 610}]}'

In [3]:
shift_rates = {"medic": 305, "intern": 142}
multiple = {"2017-1-1": 2, 
           "2017-1-2": 1,
           "2017-1-3": 1,
           "2017-1-4": 1,
           "2017-1-5": 1,
           "2017-1-6": 1,
           "2017-1-7": 2,
           "2017-1-8": 2,
           "2017-1-9": 1,
           "2017-1-10": 1,}

def retrieve_personel_id_pay2(json_file):
    # Get the data in input.json (as a dictionary)
    f = open(json_file)
    data = json.load(f)
    f.close()

    # Convert data["workers"] into a dataframe
    df = pd.DataFrame(data["workers"])
    shifts = pd.DataFrame(data["shifts"])
    
    # Add new "Work dates" and "pay" columns
    df["Work dates"] = np.nan
    df["pay"] = 0
    
    for index1, row1 in df.iterrows():
        staff_id = row1["id"]
        for index2, row2 in shifts.iterrows():
            if row2["user_id"] == staff_id:
                if df.isna().loc[index1, "Work dates"]:
                    df.loc[index1, "Work dates"] = [row2["start_date"]]
                else:
                    df.loc[index1, "Work dates"].append(row2["start_date"])
        pay_per_shift = [shift_rates[row1["status"]] * multiple[df.loc[index1, "Work dates"][i]] for i in range(len(df.loc[index1, "Work dates"]))]
        df.loc[index1,"pay"] = sum(pay_per_shift)
        
    # Filter the columns of interest
    df = df[["id", "pay"]]
    
    # Convert data back into a dictionary
    output_dict = {}
    output_dict["workers"] = df.to_dict("records")
    
    # Convert data back into a json file
    with open("output2.json", "w") as f:
        json.dump(output_dict, f)
    
    return json.dumps(output_dict)

retrieve_personel_id_pay2("input.json")

'{"workers": [{"id": 1, "pay": 1220}, {"id": 2, "pay": 915}, {"id": 3, "pay": 426}, {"id": 4, "pay": 915}]}'

## Task 3
Our hospital shift manager model is to price our service 5% of each shift. In addition to that we also help hospitals to fill in unassigned shifts with interim workers. An interim is paid USD543 per shift. We also take a fixed fee of USD90 on each shift for which we provided an interim worker.

Adapt the price computation to take these new rules into account and to calculate our commission. Look in data_interim.json for example input. Expected output can be found in output_interim.json

In [14]:
shift_rates = {"medic": 305, "intern": 142, "interim": 543}
multiple = {"2017-1-1": 2, 
           "2017-1-2": 1,
           "2017-1-3": 1,
           "2017-1-4": 1,
           "2017-1-5": 1,
           "2017-1-6": 1,
           "2017-1-7": 2,
           "2017-1-8": 2,
           "2017-1-9": 1,
           "2017-1-10": 1,
           "2017-1-11": 1}

def retrieve_personel_id_pay3(json_file):
    # Get the data in input.json (as a dictionary)
    f = open(json_file)
    data = json.load(f)
    f.close()

    # Convert data["workers"] into a dataframe
    df = pd.DataFrame(data["workers"])
    shifts = pd.DataFrame(data["shifts"])
    print(df)
    
    # Add new "Work dates" and "pay" columns
    df["Work dates"] = np.nan
    df["pay"] = 0
    commission = 0
    interim_shifts = 0
    
    for index1, row1 in df.iterrows():
        staff_id = row1["id"]
        for index2, row2 in shifts.iterrows():
            if row2["user_id"] == staff_id:
                if df.isna().loc[index1, "Work dates"]:
                    df.loc[index1, "Work dates"] = [row2["start_date"]]
                else:
                    df.loc[index1, "Work dates"].append(row2["start_date"])
        if row1["status"] == "interim":
            df.loc[index1,"pay"] = shift_rates[row1["status"]] * len(df.loc[index1, "Work dates"])
            commission += 90 * len(df.loc[index1, "Work dates"])
            interim_shifts += len(df.loc[index1, "Work dates"])
        else:
            pay_per_shift = [shift_rates[row1["status"]] * multiple[df.loc[index1, "Work dates"][i]] for i in range(len(df.loc[index1, "Work dates"]))]
            df.loc[index1,"pay"] = sum(pay_per_shift)
        
    # Filter the columns of interest
    df = df[["id", "pay"]]
    
    print(df)
    
    # Finalize commission calculation
    commission += df["pay"].sum()*0.05
    
    # Convert data back into a dictionary
    output_dict = {}
    output_dict["workers"] = df.to_dict("records")
    output_dict["commission"] = {"pdg_fee": commission, 
                                 "interim_shifts": interim_shifts}
    
    # Convert data back into a json file
    with open("output3.json", "w") as f:
        json.dump(output_dict, f)
    
    return json.dumps(output_dict)

retrieve_personel_id_pay3("input2.json")

   id first_name   status
0   1      Julie    medic
1   2       Marc    medic
2   3    Antoine   intern
3   4     Emilie    medic
4   5        Lea  interim
5   6        Bob   intern
   id   pay
0   1   610
1   2   915
2   3   142
3   4   915
4   5  1629
5   6   142


'{"workers": [{"id": 1, "pay": 610}, {"id": 2, "pay": 915}, {"id": 3, "pay": 142}, {"id": 4, "pay": 915}, {"id": 5, "pay": 1629}, {"id": 6, "pay": 142}], "commission": {"pdg_fee": 487.65, "interim_shifts": 3}}'

## Task 4
Our hospital coordinator would like to recognize the hardwork of each employee. They would like an automated way to identify the hardest working medic, intern, and interim. On the basis of "total pay received", construct a JSON file called hardworkers.json.

In [36]:
shift_rates = {"medic": 305, "intern": 142, "interim": 543}
multiple = {"2017-1-1": 2, 
           "2017-1-2": 1,
           "2017-1-3": 1,
           "2017-1-4": 1,
           "2017-1-5": 1,
           "2017-1-6": 1,
           "2017-1-7": 2,
           "2017-1-8": 2,
           "2017-1-9": 1,
           "2017-1-10": 1,
           "2017-1-11": 1,
           "2017-1-12": 1,
           "2017-1-13": 1,
           "2017-1-14": 2,
           "2017-1-15": 2}

def hardworkers(json_file):
    # Get the data in input.json (as a dictionary)
    f = open(json_file)
    data = json.load(f)
    f.close()

    # Convert data["workers"] into a dataframe
    df = pd.DataFrame(data["workers"])
    shifts = pd.DataFrame(data["shifts"])
    
    # Add new "Work dates" and "pay" columns
    df["Work dates"] = np.nan
    df["pay"] = 0
    
    for index1, row1 in df.iterrows():
        staff_id = row1["id"]
        for index2, row2 in shifts.iterrows():
            if row2["user_id"] == staff_id:
                if df.isna().loc[index1, "Work dates"]:
                    df.loc[index1, "Work dates"] = [row2["start_date"]]
                else:
                    df.loc[index1, "Work dates"].append(row2["start_date"])
        if row1["status"] == "interim":
            df.loc[index1,"pay"] = shift_rates[row1["status"]] * len(df.loc[index1, "Work dates"])
        else:
            pay_per_shift = [shift_rates[row1["status"]] * multiple[df.loc[index1, "Work dates"][i]] for i in range(len(df.loc[index1, "Work dates"]))]
            df.loc[index1,"pay"] = sum(pay_per_shift)
    
    # Retrieve hardworkers per category
    series = df.groupby(["status"])["pay"].transform(max) == df["pay"]
    series = series[series == True]
    hardworker_index = series.index
    df = df.loc[hardworker_index].drop(columns=["id", "pay", "Work dates"])
    
    # Convert data into a dictionary
    df = df.set_index("status")
    output_dict = df.to_dict("index")
    output_dict = {key: output_dict[key]["first_name"] for key in output_dict.keys()}
    
    # Convert data back into a json file
    with open("output3.json", "w") as f:
        json.dump(output_dict, f)
    
    return json.dumps(output_dict)

hardworkers("input2.json")

'{"medic": "Emilie", "interim": "Lea", "intern": "Bob"}'